### Import libraries

In [228]:
from model import climate_model, losses, dot_prod_attention
from data import data_generation, data_combine, batch_creator, gp_kernels
from keras.callbacks import ModelCheckpoint
from helpers import helpers, masks
from inference import infer
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np
import matplotlib 
import time
import keras
np.random.seed(1)

In [216]:
save_dir = '/Users/omernivron/Downloads/GPT_climate'

In [187]:
temp, t, token = data_combine.climate_data_to_model_input('./data/t2m_monthly_averaged_ensemble_members_1989_2019.csv')

In [190]:
## create climate train/test split

In [210]:
time_tr = t[:8000]; temp_tr = temp[:8000]; token_tr = token[:8000]
time_te = t[8000:]; temp_te = temp[8000:]; token_te = token[8000:]

In [229]:
loss_object = tf.keras.losses.MeanSquaredError()
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')
m_tr = tf.keras.metrics.Mean()
m_te = tf.keras.metrics.Mean()

In [230]:
@tf.function
def train_step(decoder, optimizer_c, token_pos, time_pos, tar, pos_mask):
    '''
    A typical train step function for TF2. Elements which we wish to track their gradient
    has to be inside the GradientTape() clause. see (1) https://www.tensorflow.org/guide/migrate 
    (2) https://www.tensorflow.org/tutorials/quickstart/advanced
    ------------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask (np array): see description in position_mask function
    ------------------    
    '''
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    combined_mask_tar = masks.create_masks(tar_inp)
    with tf.GradientTape(persistent=True) as tape:
        pred, pred_sig = decoder(token_pos, time_pos, tar_inp, True, pos_mask, combined_mask_tar)
#         print('pred: ')
#         tf.print(pred_sig)

        loss, mse, mask = losses.loss_function(tar_real, pred, pred_sig)


    gradients = tape.gradient(loss, decoder.trainable_variables)
#     tf.print(gradients)
# Ask the optimizer to apply the processed gradients.
    optimizer_c.apply_gradients(zip(gradients, decoder.trainable_variables))
    train_loss(loss)
    m_tr.update_state(mse, mask)
#     b = decoder.trainable_weights[0]
#     tf.print(tf.reduce_mean(b))
    return tar_inp, tar_real, pred, pred_sig, mask

In [231]:
@tf.function
def test_step(decoder, token_pos_te, time_pos_te, tar_te, pos_mask_te):
    '''
    
    ---------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask_te (np array): see description in position_mask function
    ---------------
    
    '''
    tar_inp_te = tar_te[:, :-1]
    tar_real_te = tar_te[:, 1:]
    combined_mask_tar_te = masks.create_masks(tar_inp_te)
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    pred_te, pred_sig_te = decoder(token_pos_te, time_pos_te, tar_inp_te, False, pos_mask_te, combined_mask_tar_te)
    t_loss, t_mse, t_mask = losses.loss_function(tar_real_te, pred_te, pred_sig_te)
    test_loss(t_loss)
    m_te.update_state(t_mse, t_mask)
    return tar_real_te, pred_te, pred_sig_te, t_mask

In [232]:
tf.keras.backend.set_floatx('float64')

In [ ]:
if __name__ == '__main__':
    writer = tf.summary.create_file_writer(save_dir + '/logs/')
    optimizer_c = tf.keras.optimizers.Adam()
    decoder = climate_model.Decoder(16)
    EPOCHS = 500
    batch_s  = 32
    run = 0; step = 0
    num_batches = int(temp_tr.shape[0] / batch_s)
    tf.random.set_seed(1)
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer = optimizer_c, net = decoder)
    main_folder = "/Users/omernivron/Downloads/GPT_climate/ckpt/check_"
    folder = main_folder + str(run); helpers.mkdir(folder)
    #https://www.tensorflow.org/guide/checkpoint
    manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=3)
    ckpt.restore(manager.latest_checkpoint)
    if manager.latest_checkpoint:
        print("Restored from {}".format(manager.latest_checkpoint))
    else:
        print("Initializing from scratch.")

    with writer.as_default():
        for epoch in range(EPOCHS):
            start = time.time()

            for batch_n in range(num_batches):
                batch_tok_pos_tr, batch_tim_pos_tr, batch_tar_tr, _ = batch_creator.create_batch_foxes(token_tr, time_tr, temp_tr, batch_s=32)
                # batch_tar_tr shape := 128 X 59 = (batch_size, max_seq_len)
                # batch_pos_tr shape := 128 X 59 = (batch_size, max_seq_len)
                batch_pos_mask = masks.position_mask(batch_tok_pos_tr)
                tar_inp, tar_real, pred, pred_sig, mask = train_step(decoder, optimizer_c, batch_tok_pos_tr, batch_tim_pos_tr, batch_tar_tr, batch_pos_mask)

                if batch_n % 10 == 0:
                    batch_tok_pos_te, batch_tim_pos_te, batch_tar_te, _ = batch_creator.create_batch_foxes(token_te, time_te, temp_te, batch_s= 32)
                    batch_pos_mask_te = masks.position_mask(batch_tok_pos_te)
                    tar_real_te, pred_te, pred_sig_te, t_mask = test_step(decoder, batch_tok_pos_te, batch_tim_pos_te, batch_tar_te, batch_pos_mask_te)
                    helpers.print_progress(epoch, batch_n, train_loss.result(), test_loss.result(), m_tr.result())
                    helpers.tf_summaries(run, step, train_loss.result(), test_loss.result(), m_tr.result(), m_te.result())
                    manager.save()
                step += 1
                ckpt.step.assign_add(1)

            print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Already exists
Initializing from scratch.
Epoch 0 batch 0 train Loss 276635.7271 test Loss 92122.0270 with MSE metric 45551.5664
Epoch 0 batch 10 train Loss 147707.3831 test Loss 46122.4647 with MSE metric 46528.5883
Epoch 0 batch 20 train Loss 88084.5578 test Loss 30754.6986 with MSE metric 46206.7365
Epoch 0 batch 30 train Loss 62429.0003 test Loss 23068.2816 with MSE metric 45921.8479
Epoch 0 batch 40 train Loss 48274.3682 test Loss 18456.1269 with MSE metric 46211.5025
Epoch 0 batch 50 train Loss 39442.4337 test Loss 15381.2496 with MSE metric 46455.9027
Epoch 0 batch 60 train Loss 33508.3575 test Loss 13184.8707 with MSE metric 46346.1084
Epoch 0 batch 70 train Loss 29167.5388 test Loss 11537.5780 with MSE metric 46335.6242
Epoch 0 batch 80 train Loss 25797.9345 test Loss 10256.3357 with MSE metric 46273.0657
Epoch 0 batch 90 train Loss 23146.2318 test Loss 9231.3473 with MSE metric 46227.9679
Epoch 0 batch 100 train Loss 21060.0951 test Loss 8392.7229 with MSE metric 46165.1620
E

Epoch 3 batch 190 train Loss 2573.7582 test Loss 978.2907 with MSE metric 45097.1019
Epoch 3 batch 200 train Loss 2547.3261 test Loss 968.1856 with MSE metric 45086.2424
Epoch 3 batch 210 train Loss 2521.5919 test Loss 958.2892 with MSE metric 45083.5910
Epoch 3 batch 220 train Loss 2496.2625 test Loss 948.5948 with MSE metric 45077.8469
Epoch 3 batch 230 train Loss 2471.7906 test Loss 939.0961 with MSE metric 45075.7935
Epoch 3 batch 240 train Loss 2447.3417 test Loss 929.7878 with MSE metric 45074.1080
Time taken for 1 epoch: 30.357666015625 secs

Epoch 4 batch 0 train Loss 2423.4229 test Loss 920.6635 with MSE metric 45070.6918
Epoch 4 batch 10 train Loss 2399.9280 test Loss 911.7182 with MSE metric 45057.1591
Epoch 4 batch 20 train Loss 2376.9605 test Loss 902.9480 with MSE metric 45060.1760
Epoch 4 batch 30 train Loss 2354.4834 test Loss 894.3457 with MSE metric 45053.5382
Epoch 4 batch 40 train Loss 2332.3808 test Loss 885.9081 with MSE metric 45056.1516
Epoch 4 batch 50 train Lo

Epoch 7 batch 140 train Loss 1300.7095 test Loss 493.4961 with MSE metric 44818.2943
Epoch 7 batch 150 train Loss 1294.0040 test Loss 490.9602 with MSE metric 44817.6849
Epoch 7 batch 160 train Loss 1287.3511 test Loss 488.4506 with MSE metric 44813.3637
Epoch 7 batch 170 train Loss 1280.7618 test Loss 485.9669 with MSE metric 44810.6646
Epoch 7 batch 180 train Loss 1274.3010 test Loss 483.5091 with MSE metric 44811.2059
Epoch 7 batch 190 train Loss 1267.8261 test Loss 481.0765 with MSE metric 44810.5494
Epoch 7 batch 200 train Loss 1261.4127 test Loss 478.6689 with MSE metric 44809.8589
Epoch 7 batch 210 train Loss 1255.0965 test Loss 476.2854 with MSE metric 44813.5021
Epoch 7 batch 220 train Loss 1248.8292 test Loss 473.9266 with MSE metric 44812.7825
Epoch 7 batch 230 train Loss 1242.6034 test Loss 471.5913 with MSE metric 44808.9969
Epoch 7 batch 240 train Loss 1236.4422 test Loss 469.2792 with MSE metric 44808.6132
Time taken for 1 epoch: 29.421195030212402 secs

Epoch 8 batch 0 

Epoch 11 batch 90 train Loss 871.8436 test Loss 332.1477 with MSE metric 44707.9802
Epoch 11 batch 100 train Loss 868.8273 test Loss 331.0205 with MSE metric 44706.4711
Epoch 11 batch 110 train Loss 865.8288 test Loss 329.9011 with MSE metric 44702.7393
Epoch 11 batch 120 train Loss 862.8777 test Loss 328.7893 with MSE metric 44698.5043
Epoch 11 batch 130 train Loss 859.9492 test Loss 327.6854 with MSE metric 44698.4619
Epoch 11 batch 140 train Loss 857.0172 test Loss 326.5892 with MSE metric 44696.4532
Epoch 11 batch 150 train Loss 854.1239 test Loss 325.5003 with MSE metric 44692.4952
Epoch 11 batch 160 train Loss 851.2339 test Loss 324.4191 with MSE metric 44690.6223
Epoch 11 batch 170 train Loss 848.3575 test Loss 323.3452 with MSE metric 44689.6701
Epoch 11 batch 180 train Loss 845.5189 test Loss 322.2788 with MSE metric 44688.6152
Epoch 11 batch 190 train Loss 842.7060 test Loss 321.2197 with MSE metric 44688.2442
Epoch 11 batch 200 train Loss 839.8990 test Loss 320.1676 with MSE

Epoch 15 batch 40 train Loss 656.4897 test Loss 251.6091 with MSE metric 44649.6013
Epoch 15 batch 50 train Loss 654.7873 test Loss 250.9754 with MSE metric 44650.0012
Epoch 15 batch 60 train Loss 653.1027 test Loss 250.3452 with MSE metric 44650.0421
Epoch 15 batch 70 train Loss 651.4181 test Loss 249.7183 with MSE metric 44649.1121
Epoch 15 batch 80 train Loss 649.7408 test Loss 249.0947 with MSE metric 44647.7067
Epoch 15 batch 90 train Loss 648.0725 test Loss 248.4744 with MSE metric 44646.1725
Epoch 15 batch 100 train Loss 646.4165 test Loss 247.8572 with MSE metric 44645.2851
Epoch 15 batch 110 train Loss 644.7653 test Loss 247.2429 with MSE metric 44644.8591
Epoch 15 batch 120 train Loss 643.1229 test Loss 246.6321 with MSE metric 44643.1733
Epoch 15 batch 130 train Loss 641.4899 test Loss 246.0245 with MSE metric 44641.2036
Epoch 15 batch 140 train Loss 639.8646 test Loss 245.4198 with MSE metric 44639.2454
Epoch 15 batch 150 train Loss 638.2462 test Loss 244.8183 with MSE metr

...

Epoch 234 batch 30 train Loss 47.9756 test Loss 22.4297 with MSE metric 18104.8516
Epoch 234 batch 40 train Loss 47.9683 test Loss 22.4268 with MSE metric 18103.0342
Epoch 234 batch 50 train Loss 47.9610 test Loss 22.4239 with MSE metric 18101.3285
Epoch 234 batch 60 train Loss 47.9537 test Loss 22.4210 with MSE metric 18099.5609
Epoch 234 batch 70 train Loss 47.9464 test Loss 22.4181 with MSE metric 18097.7139
Epoch 234 batch 80 train Loss 47.9392 test Loss 22.4152 with MSE metric 18095.9320
Epoch 234 batch 90 train Loss 47.9319 test Loss 22.4123 with MSE metric 18094.1762
Epoch 234 batch 100 train Loss 47.9246 test Loss 22.4094 with MSE metric 18092.3733
Epoch 234 batch 110 train Loss 47.9173 test Loss 22.4065 with MSE metric 18090.5772
Epoch 234 batch 120 train Loss 47.9101 test Loss 22.4036 with MSE metric 18088.8113
Epoch 234 batch 130 train Loss 47.9028 test Loss 22.4006 with MSE metric 18086.9855
Epoch 234 batch 140 train Loss 47.8955 test Loss 22.3977 with MSE metric 18085.1560

Time taken for 1 epoch: 30.929280996322632 secs

Epoch 238 batch 0 train Loss 47.2801 test Loss 22.1520 with MSE metric 17934.2647
Epoch 238 batch 10 train Loss 47.2731 test Loss 22.1492 with MSE metric 17932.4923
Epoch 238 batch 20 train Loss 47.2660 test Loss 22.1464 with MSE metric 17930.7743
Epoch 238 batch 30 train Loss 47.2590 test Loss 22.1435 with MSE metric 17929.0491
Epoch 238 batch 40 train Loss 47.2519 test Loss 22.1407 with MSE metric 17927.3859
Epoch 238 batch 50 train Loss 47.2449 test Loss 22.1379 with MSE metric 17925.6194
Epoch 238 batch 60 train Loss 47.2378 test Loss 22.1351 with MSE metric 17923.8536
Epoch 238 batch 70 train Loss 47.2308 test Loss 22.1323 with MSE metric 17922.2029
Epoch 238 batch 80 train Loss 47.2238 test Loss 22.1295 with MSE metric 17920.5072
Epoch 238 batch 90 train Loss 47.2167 test Loss 22.1267 with MSE metric 17918.7359
Epoch 238 batch 100 train Loss 47.2097 test Loss 22.1239 with MSE metric 17916.9645
Epoch 238 batch 110 train Loss 47.2027

Epoch 241 batch 210 train Loss 46.6137 test Loss 21.8860 with MSE metric 17770.1446
Epoch 241 batch 220 train Loss 46.6069 test Loss 21.8833 with MSE metric 17768.4386
Epoch 241 batch 230 train Loss 46.6001 test Loss 21.8806 with MSE metric 17766.7777
Epoch 241 batch 240 train Loss 46.5933 test Loss 21.8778 with MSE metric 17765.0795
Time taken for 1 epoch: 29.473978281021118 secs

Epoch 242 batch 0 train Loss 46.5864 test Loss 21.8751 with MSE metric 17763.3826
Epoch 242 batch 10 train Loss 46.5796 test Loss 21.8724 with MSE metric 17761.7321
Epoch 242 batch 20 train Loss 46.5728 test Loss 21.8696 with MSE metric 17760.0578
Epoch 242 batch 30 train Loss 46.5660 test Loss 21.8669 with MSE metric 17758.3264
Epoch 242 batch 40 train Loss 46.5592 test Loss 21.8642 with MSE metric 17756.6753
Epoch 242 batch 50 train Loss 46.5523 test Loss 21.8615 with MSE metric 17754.9663
Epoch 242 batch 60 train Loss 46.5455 test Loss 21.8588 with MSE metric 17753.3582
Epoch 242 batch 70 train Loss 46.53

Epoch 245 batch 170 train Loss 45.9682 test Loss 21.6282 with MSE metric 17611.1689
Epoch 245 batch 180 train Loss 45.9616 test Loss 21.6255 with MSE metric 17609.5246
Epoch 245 batch 190 train Loss 45.9550 test Loss 21.6229 with MSE metric 17607.9538
Epoch 245 batch 200 train Loss 45.9484 test Loss 21.6202 with MSE metric 17606.3292
Epoch 245 batch 210 train Loss 45.9418 test Loss 21.6176 with MSE metric 17604.7086
Epoch 245 batch 220 train Loss 45.9352 test Loss 21.6150 with MSE metric 17603.0676
Epoch 245 batch 230 train Loss 45.9285 test Loss 21.6124 with MSE metric 17601.3933
Epoch 245 batch 240 train Loss 45.9219 test Loss 21.6097 with MSE metric 17599.7641
Time taken for 1 epoch: 27.911991119384766 secs

Epoch 246 batch 0 train Loss 45.9153 test Loss 21.6071 with MSE metric 17598.1434
Epoch 246 batch 10 train Loss 45.9087 test Loss 21.6044 with MSE metric 17596.4817
Epoch 246 batch 20 train Loss 45.9021 test Loss 21.6018 with MSE metric 17594.8499
Epoch 246 batch 30 train Loss 4

Epoch 249 batch 130 train Loss 45.3426 test Loss 21.3781 with MSE metric 17457.5049
Epoch 249 batch 140 train Loss 45.3362 test Loss 21.3756 with MSE metric 17455.9776
Epoch 249 batch 150 train Loss 45.3298 test Loss 21.3730 with MSE metric 17454.3847
Epoch 249 batch 160 train Loss 45.3234 test Loss 21.3705 with MSE metric 17452.7957
Epoch 249 batch 170 train Loss 45.3169 test Loss 21.3679 with MSE metric 17451.1789
Epoch 249 batch 180 train Loss 45.3105 test Loss 21.3653 with MSE metric 17449.6354
Epoch 249 batch 190 train Loss 45.3041 test Loss 21.3628 with MSE metric 17448.0767
Epoch 249 batch 200 train Loss 45.2977 test Loss 21.3602 with MSE metric 17446.4880
Epoch 249 batch 210 train Loss 45.2913 test Loss 21.3576 with MSE metric 17444.8587
Epoch 249 batch 220 train Loss 45.2849 test Loss 21.3551 with MSE metric 17443.2754
Epoch 249 batch 230 train Loss 45.2785 test Loss 21.3525 with MSE metric 17441.7247
Epoch 249 batch 240 train Loss 45.2721 test Loss 21.3500 with MSE metric 174

In [ ]:
1 - (0.0165 / sum((tar[:, 5] - np.mean(tar[:, 5]))**2) / len(tar[:, 5]))

In [ ]:
tar - np.mean(tar, 0)

In [ ]:
tar.shape

In [ ]:
np.mean(tar[:, 0])

In [ ]:
sum((tar[:, 0] - np.mean(tar[:, 0]))**2 )/ 10000

In [ ]:
sum(sum((tar - np.mean(tar))**2)) / (tar.shape[0] * tar.shape[1])

In [ ]:
pos = df_te[560, :].reshape(1, -1)

In [ ]:
tar = df_te[561, :39].reshape(1, -1)

In [ ]:
df_te[561, :]

In [ ]:
a = inference(pos, tar, 20)

In [ ]:
with matplotlib.rc_context({'figure.figsize': [10,2.5]}):
    plt.scatter(pos[:, :39], tar[:, :39], c='black')
    plt.scatter(pos[:, 39:58], a[39:])
    plt.scatter(pos[:, 39:58], df_te[561, 39:58], c='red')

In [ ]:
# tf.data.Dataset(tf.Tensor(pad_pos_tr, value_index = 0 , dtype = tf.float32))